# Tuning Script

In [1]:
# Parameters subject to change in every script - involved in naming

MODEL = 'tf'
TUNER = 'jiaochengb'

---

In [2]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Mounted at /content/drive


In [3]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cbr':
        !pip install catboost
    elif MODEL in ['tf', 'mlp']:
        !pip install PyTorch2Sklearn==0.0.13
    !pip install JXAutoML==0.2.8
else:
    home_directory = '../../'

from JXAutoML.JiaoChengB import JiaoChengB as Tuner

import pandas as pd

In [4]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = True

PyTorchModel = True

---

In [5]:
# model to use to tune
from PyTorch2Sklearn.Transformer import Transformer as clf
from PyTorch2Sklearn.environment import *
from PyTorch2Sklearn.utils.data import TabularDataFactory, TabularDataset


# what values to try for each hyperparameter
parameter_choices = {
    'hidden_dim': (16, 32, 64, 128, 256),
    'num_transformer_layers': (1, 2, 3),
    'num_mlp_layers': (1, 2, 3),
    'dropout': (0, 0.05, 0.1, 0.2),
    'batch_size': (32, 128, 512, 1024, 2048, 4096),
    'nhead': (2, 4, 8, 16),
    'share_embedding_mlp': (False, True),
    'use_cls': (False, True),
    'epochs': (5, 10, 20, 30),
    'lr': (1e-5, 1e-4, 1e-3, 1e-2, 1e-1),
    'batchnorm': (False, True),
    'grad_clip': (False, True),
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
    'random_state': SEED,
    'loss': nn.MSELoss(),
    'rootpath': './',
    'TabularDataFactory': TabularDataFactory,
    'TabularDataset': TabularDataset,
    'name': 'Transformer',
    'output_dim': 1,
    'mode': 'Regression',
    'verbose': True,
}

# what order to tune hyperparameters in (JiaoCheng specific)
tuning_order = ['features',
                'num_transformer_layers',
                'num_mlp_layers',
                'nhead',
                'hidden_dim',
                'dropout',
                'share_embedding_mlp',
                'use_cls',
                'batchnorm',
                'grad_clip',
                'lr',
                'epochs',
                'batch_size'
                ]

# what order to tune hyperparameters in (JiaoCheng specific)
default_hyperparameter_values = {
    'features': 36,
    'num_transformer_layers': 1,
    'num_mlp_layers': 1,
    'nhead': 4,
    'hidden_dim': 16,
    'dropout': 0.1,
    'share_embedding_mlp': True,
    'use_cls': False,
    'batchnorm': False,
    'grad_clip': False,
    'lr': 1e-2,
    'epochs': 10,
    'batch_size': 128
}

---

Read in and Prepare Data

In [6]:
train_data = pd.read_csv(f'{home_directory}data/curated/modelling/train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/val.csv')
test_data = pd.read_csv(f'{home_directory}data/curated/modelling/test.csv')

train_X = train_data.drop(columns=['target'])
train_y = train_data['target']
val_X = val_data.drop(columns=['target'])
val_y = val_data['target']
test_X = test_data.drop(columns=['target'])
test_y = test_data['target']

In [7]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [8]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(
    clf, TASK_TYPE, pytorch_model=PyTorchModel, optimised_metric='r2')

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up the order of hyperparameters when iteratively tuning using JiaoCheng
tuner.set_tuning_order(tuning_order)

print('---')

# set up the default hp values for first iteration of tuning JiaoCheng
tuner.set_hyperparameter_default_values(default_hyperparameter_values)

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

JiaoCheng Initialised
---
Successfully read in model <class 'PyTorch2Sklearn.Transformer.Transformer'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
Successfully recorded tuneable feature combination choices and updated relevant internal structures
---
---
---
Successfully read in tuning result of 259 rows, for 259.0 combos
---
Successfully set tuning output address
---
Successfully set best model output address


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# begin tuning ... (or continue training)

tuner.tune()

Streaming output truncated to the last 5000 lines.
        Has trained 259 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 31), which had val score of 0.9004
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 259 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 32), which had val score of 0.9009
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 259 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 33), which had val score of 0.901
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 259 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 34), which had val score of 0.9009
   

100%|██████████| 10/10 [01:18<00:00,  7.86s/it]


Trained and Tested combination 260 of 51148800: (0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 36), taking 78.81 seconds to get val score of 0.8672
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028
Already Trained and Tested combination (0, 0, 0, 2, 1, 1, 1, 0, 1, 3, 0, 0, 36), which had val score of 0.8927
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 260 of 51148800 combinations so far


100%|██████████| 10/10 [01:17<00:00,  7.79s/it]


Trained and Tested combination 261 of 51148800: (0, 0, 0, 2, 1, 1, 1, 0, 1, 4, 0, 0, 36), taking 78.17 seconds to get val score of -2.868
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028

Best combo after this hyperparameter: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) , UPDATED SINCE LAST HYPERPARAMETER


Round 1 
Hyperparameter: epochs (index: 8) 

Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 36), which had val score of 0.8979
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 261 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36), which had val score of 0.9028
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 261 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 36), which h

...

Trained and Tested combination 267 of 51148800: (0, 0, 0, 2, 3, 1, 1, 0, 1, 3, 0, 0, 36), taking 28.15 seconds to get val score of 0.8134
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028


100%|██████████| 10/10 [00:48<00:00,  4.80s/it]


Trained and Tested combination 268 of 51148800: (0, 0, 0, 2, 4, 1, 1, 0, 1, 3, 0, 0, 36), taking 48.24 seconds to get val score of 0.8496
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028


100%|██████████| 10/10 [01:30<00:00,  9.03s/it]


Trained and Tested combination 269 of 51148800: (0, 0, 0, 2, 5, 1, 1, 0, 1, 3, 0, 0, 36), taking 90.62 seconds to get val score of 0.8682
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028

Best combo after this hyperparameter: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) , UPDATED SINCE LAST HYPERPARAMETER


Round 1 
Hyperparameter: features (index: 12) 

Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0), which had val score of 0.585
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 269 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1), which had val score of 0.6059
        Current best combo: (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 36) with val score 0.9028. 
        Has trained 269 of 51148800 combinations so far
Already Trained and Tested combination (0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 2), which ha